# Показатели ТТВ Тематическое по нескольким ЦА
Пример расчета показателей Тотал ТВ Тематическое

## Описание задачи и условий расчета

- Период: сентябрь 2022
- Временной интервал: 05:00-29:00
- ЦА: Все 4+, Мужчины, Женщины
- Место просмотра: Все места (дом+дача)
- Каналы: ТТВ Тематическое									
- Статистики: Universe, TTV Rtg%, TTV AvReach%, TTV Reach%, TTV ATV

## Инициализация

При построении отчета первый шаг в любом ноутбуке - загрузка библиотек, которые помогут обращаться к TVI API и работать с данными.

Выполните следующую ячейку, для этого перейдите в нее и нажмите Ctrl+Enter

In [ ]:
%reload_ext autoreload
%autoreload 2

import sys
import os
import re
import json
import datetime
import time
import pandas as pd
from IPython.display import JSON

from mediascope_api.core import net as mscore
from mediascope_api.mediavortex import tasks as cwt
from mediascope_api.mediavortex import catalogs as cwc

# Настраиваем отображение

# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)
# Задаем максимальное количество выводимых строк. Раскомментируйте нужную строку
# 200 строк
# pd.set_option("display.max_rows", 200)
# Отображаем все строки. ВАЖНО! Отображение большого DataFrame требует много ресурсов
# pd.set_option("display.max_rows", None)

# Cоздаем объекты для работы с TVI API
mnet = mscore.MediascopeApiNetwork()
mtask = cwt.MediaVortexTask()
cats = cwc.MediaVortexCats()

## Справочники

Получим идентификаторы, которые будут использоваться для формирования условий расчета

In [ ]:
# В справочнике демографических атрибутов найдем необходимые нам переменные
cats.get_tv_demo_attribute(names=['пол', 'возраст'])

# Для формирования условий запроса нам понадобятся значения столбца entityName: sex, age

## Формирование задания
Зададим условия расчета

In [ ]:
# Период указывается в виде списка ('Начало', 'Конец') в формате 'YYYY-MM-DD'.
date_filter = [('2022-09-01', '2022-09-30')]

# Задаем дни недели
weekday_filter = None

# Задаем тип дня
daytype_filter = None

# Задаем временной интервал - минимум 60 минут
time_filter = None

# Задаем ЦА
basedemo_filter = None

# Доп фильтр ЦА, нужен только в случае расчета отношения между ЦА, например, при расчете Affinity Index
targetdemo_filter = None

# Задаем место просмотра
location_filter=None

# Задаем каналы: оставляем пустым. Тотал ТВ Тематическое задается как параметр ниже
company_filter=None

# Указываем список статистик для расчета: в данном отчете нам нужны специальные ТТВ статистики
statistics = ['Universe000', 'TTVRtgPer', 'TTVAvReachPer', 'TTVReachPer', 'TTVATV']

# Указываем срезы
slices = []

# Задаем условия сортировки
sortings = None

# Задаем опции расчета
options = {
    "kitId": 4, #TV Index Plus All Russia
    "useNbd": False, #Расчет накопленного охвата без nbd коррекции
    "totalType": "TotalChannelsThem" #база расчета тотал статистик: Total Channels Them. Возможны опции: TotalTVSet, TotalChannels
}

Сформируем словарь с целевыми аудиториями: ключ - название переменной, значение - ее синтаксис

In [ ]:
targets = {
    'Total Ind.':'age >= 4',
    'Male 4+':'sex = 1 AND age >= 4',
    'Female 4+':'sex = 2 AND age >= 4'    
}

## Расчет задания

In [ ]:
# Посчитаем задания в цикле
tasks = []
print("Отправляем задания на расчет")

# Для каждой ЦА формируем задание и отправляем на расчет
for target, syntax in targets.items():
    
    # Подставляем значения словаря в параметры 
    project_name = target
    basedemo_filter = syntax
      
    # Формируем задание для API TV Index в формате JSON
    task_json = mtask.build_timeband_task(task_name=project_name, date_filter=date_filter, 
                                          weekday_filter=weekday_filter, daytype_filter=daytype_filter, 
                                          company_filter=company_filter, time_filter=time_filter, 
                                          basedemo_filter=basedemo_filter, targetdemo_filter=targetdemo_filter,
                                          location_filter=location_filter, slices=slices, 
                                          statistics=statistics, sortings=sortings, options=options)

    # Для каждого этапа цикла формируем словарь с параметрами и отправленным заданием на расчет
    tsk = {}
    tsk['project_name'] = project_name    
    tsk['task'] = mtask.send_timeband_task(task_json)
    tasks.append(tsk)
    time.sleep(2)
    print('.', end = '')

print(f"\nid: {[i['task']['taskId'] for i in tasks]}") 
    
print('')
# Ждем выполнения
print('Ждем выполнения')
tsks = mtask.wait_task(tasks)
print('Расчет завершен, получаем результат')

# Получаем результат
results = []
print('Собираем таблицу')
for t in tasks:
    tsk = t['task'] 
    df_result = mtask.result2table(mtask.get_result(tsk), project_name = t['project_name'])        
    results.append(df_result)
    print('.', end = '')
df = pd.concat(results)

In [ ]:
# Приводим порядок столбцов в соответствие с условиями расчета
df = df[['prj_name']+slices+statistics]
df

## Экспорт в Excel
По умолчанию файл сохраняется в папку `excel`

In [ ]:
df_info = mtask.task_builder.get_report_info()

with pd.ExcelWriter(mtask.task_builder.get_excel_filename('thematic_02_TTV_them')) as writer:
    df.to_excel(writer, 'Report', index=True)
    df_info.to_excel(writer, 'Info', index=False)